<a href="https://colab.research.google.com/github/pytholabsbot1/Anime_Recommendation_system/blob/master/anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle lightfm

# !mkdir /root/.kaggle

# !ls /root/.kaggle

# open('/root/.kaggle/kaggle.json','w').write('{"username":"vaqasam","key":"2ec5d91c66a678bc314920e84c687f3b"}')
# !chmod 600 /root/.kaggle/kaggle.json


    100% |████████████████████████████████| 307kB 7.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
!kaggle datasets download CooperUnion/anime-recommendations-database

 83% 21.0M/25.2M [00:00<00:00, 56.1MB/s]
100% 25.2M/25.2M [00:00<00:00, 100MB/s] 


In [0]:
!unzip anime-recommendations-database.zip

Archive:  anime-recommendations-database.zip
  inflating: anime.csv               
  inflating: rating.csv              


In [0]:
from google.colab import files

files.upload()

Saving generic_preprocessing.py to generic_preprocessing.py
Saving recsys.py to recsys.py


{'generic_preprocessing.py': b'"""\n@author: Aayush Agrawal\n@Purpose - Re-usable code in Python 3 for general data exploration and pre-processing in modeling process\n"""\n\n## Importing required libraries\nimport pandas as pd ## For DataFrame operation\nimport numpy as np ## Numerical python for matrix operations\nfrom sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler ## Preprocessing function\nimport pandas_profiling ## For easy profiling of pandas DataFrame\nimport missingno as msno ## Missing value co-occurance analysis\n\n####### Data Exploration ############\n\ndef print_dim(df):\n    \'\'\'\n    Function to print the dimensions of a given python dataframe\n    Required Input -\n        - df = Pandas DataFrame\n    Expected Output -\n        - Data size\n    \'\'\'\n    print("Data size: Rows-{0} Columns-{1}".format(df.shape[0],df.shape[1]))\n\n\ndef print_dataunique(df):\n    \'\'\'\n    Function to print unique information for each column in a python data

In [0]:
# Importing Libraries and cookbooks
from recsys import * ## recommender system cookbook
from generic_preprocessing import * ## pre-processing code
from IPython.display import HTML ## Setting display options for Ipython Notebook
import pandas as pd,numpy as np

In [22]:
anime = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv').iloc[:5000000]

ids = ratings.anime_id.unique()

anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
for i,el in enumerate(anime.anime_id):
  if(el in ids):
    print(el)
    break

32281


In [23]:
# Creating interaction matrix using rating data
interactions = create_interaction_matrix(df = ratings,
                                         user_col = 'user_id',
                                         item_col = 'anime_id',
                                         rating_col = 'rating')
interactions.shape

(47848, 9927)

In [0]:
sum(ratings.user_id==3)

94

In [0]:
# Create User Dict
user_dict = create_user_dict(interactions=interactions)
# Create Item dict
movies_dict = create_item_dict(df = anime,
                               id_col = 'anime_id',
                               name_col = 'name')

In [0]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [0]:
## Creating item-item distance matrix
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,
                                                       interactions = interactions)
## Checking item embedding distance matrix
#item_item_dist.head()


In [31]:
## Calling 10 recommended items for item id 
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 8363,
                                    item_dict = movies_dict,
                                    n_items = 10)

Item of interest :Crayon Shin-chan Movie 11: Arashi wo Yobu Eikou no Yakiniku Road
Item similar to the above item:
1- Crayon Shin-chan Movie 07: Bakuhatsu! Onsen Wakuwaku Daikessen
2- Crayon Shin-chan Movie 10: Arashi wo Yobu Appare! Sengoku Daikassen
3- Crayon Shin-chan Movie 12: Arashi wo Yobu! Yuuhi no Kasukabe Boys
4- Crayon Shin-chan Movie 05: Ankoku Tamatama Daitsuiseki
5- Crayon Shin-chan Movie 08: Arashi wo Yobu Jungle
6- Crayon Shin-chan Movie 06: Dengeki! Buta no Hizume Daisakusen
7- Crayon Shin-chan Movie 13: Densetsu wo Yobu Buriburi 3 Pun Dai Shingeki
8- Crayon Shin-chan Movie 03: Unkokusai no Yabou
9- Crayon Shin-chan Movie 15: Arashi wo Yobu Utau Ketsu dake Bakudan!
10- Crayon Shin-chan Movie 16: Chou Arashi wo Yobu Kinpoko no Yuusha


In [0]:
l = anime.name[ratings.anime_id.unique()]

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [18]:
q = input('anime key: ')

for i in movies_dict:
  if(q in movies_dict[i].lower()):
    print(i,movies_dict[i])

anime key: shin-chan
2450 Crayon Shin-chan Movie 09: Arashi wo Yobu Mouretsu! Otona Teikoku no Gyakushuu
966 Crayon Shin-chan
3744 Crayon Shin-chan Movie 10: Arashi wo Yobu Appare! Sengoku Daikassen
21395 Crayon Shin-chan Movie 22: Gachinko! Gyakushuu no Robo To-chan
8369 Crayon Shin-chan Movie 18: Chou Jikuu! Arashi wo Yobu Ora no Hanayome
6460 Crayon Shin-chan Movie 05: Ankoku Tamatama Daitsuiseki
8359 Crayon Shin-chan Movie 04: Henderland no Daibouken
8363 Crayon Shin-chan Movie 11: Arashi wo Yobu Eikou no Yakiniku Road
8364 Crayon Shin-chan Movie 12: Arashi wo Yobu! Yuuhi no Kasukabe Boys
8362 Crayon Shin-chan Movie 08: Arashi wo Yobu Jungle
6217 Crayon Shin-chan Movie 03: Unkokusai no Yabou
3745 Crayon Shin-chan Movie 02: Buriburi Oukoku no Hihou
8358 Crayon Shin-chan Movie 01: Action Kamen vs. Haigure Maou
8360 Crayon Shin-chan Movie 06: Dengeki! Buta no Hizume Daisakusen
2172 Crayon Shin-chan Movie 15: Arashi wo Yobu Utau Ketsu dake Bakudan!
27653 Crayon Shin-chan Movie 23: Ora 